# Report for Homework 1: Classifier Agent

### Name:
### Panther ID:


## Declaration of Sources and Collaboration:

You should declare who you worked with in this homework. Don't submit identical works, rather study together and apply it individually.

## Part 1:  Gradient Calculations

The loss function to use is the cross-entropy loss, averaged over data points.

You should include your work for deriving the full gradient and stochastic gradient here.

You can type equations using LATEX in markdown with, e.g. $
e = mc^2, v = \frac{dx}{dt}
$

## Part 2:  Gradient Descent vs Stochastic Gradient Descent

Discuss what do you observe about GD and SGD from your implementation? Which one is faster in terms of number of iterations, which one is faster in terms of the wall clock time?

You can plot the learning curves, e.g., training error against epochs and wall-clock time.

You can use matplotlib for plotting such figures.

## Part 3: Apply the model to your own text

In [2]:
from classifier import load_data,tokenize, compute_word_idf
from classifier import custom_feature_extractor, classifier_agent
import numpy as np

# First load the classifier

with open('data/vocab.txt') as file:
    reading = file.readlines()
    vocab_list = [item.strip() for item in reading]

feat_map = custom_feature_extractor(vocab_list, tokenize)

d = len(vocab_list)
params = np.array([0.0 for i in range(d)])
custom_classifier = classifier_agent(feat_map, params)
# custom_classifier.load_params_from_file('best_model.npy')

In [2]:
# Try it out!

my_sentence = "This movie is amazing! Truly a masterpiece."

my_sentence2 = "The book is really, really good. The movie is just dreadful."

ypred = custom_classifier.predict(my_sentence,RAW_TEXT=True)

ypred2 = custom_classifier.predict(my_sentence2,RAW_TEXT=True)

print(ypred,ypred2)


[ True] [False]


### We can also try predicting for each word in the input so as to get a sense of how the classifier arrived at the prediction

In [3]:
import pandas as pd

# function for set text color of positive
# values in Dataframes
def color_predictions(val):
    eps = 0.02
    if isinstance(val,float):
        if val > eps:
            color = 'blue'
        elif val < -eps:
            color = 'red'
        else:
            color = 'black'
    else:
        color='black'
    return 'color: %s' % color

my_sentence_list = tokenize(my_sentence2)
ypred_per_word = custom_classifier.predict(my_sentence_list,RAW_TEXT=True,RETURN_SCORE=True)

df = pd.DataFrame([my_sentence_list,ypred_per_word])

df.style.applymap(color_predictions)

,0,1,2,3,4,5,6,7,8,9,10
0,the,book,is,really,really,good,the,movie,is,just,dreadful
1,-0.017113,-0.197179,0.025361,0.032491,0.032491,0.110763,-0.017113,0.047433,0.025361,-0.094084,-0.132908


### Answer the questions: 
1. Are the above results making intuitive sense and why?
2. What are some limitation of a linear classifier with BoW features?
3. what are some ideas you can come up with to overcome these limitations (i.e., what are your ideas of constructing informative features)?


## Part 4: Document what you did for custom feature extractors 

What did you try? What were the accuracy you got. What worked better and what not, and why?

## Part 5:  Anything else you'd like to write about. Your instructor / TA will read them.

You may answer for instance:

- What have you learned from the experience of working on this coding project?

- Do you think it is easy / hard? If you find it to be hard, what is the main missing piece that you think the instructor / TA should cover in the lectures / discussion sections.